In [1]:
!pip install opencv-python-headless

In [2]:

from typing import List
import itertools
import os
import shutil
import uuid
from collections import Counter
from datetime import datetime, timedelta
from pathlib import Path
import subprocess
import tempfile
import time
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec

import cv2

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12,8

import getpass
import azure.storage.blob
from azure.storage.blob import BlobClient, BlobServiceClient
from azure.core.exceptions import ResourceExistsError, HttpResponseError

In [3]:
###################################
# Read all meta data into single df
###################################

models = ["GISS_E2_1_G_ssp585", "GISS_E2_1_G_ssp245", "GFDL_ESM4_ssp245", "GFDL_ESM4_ssp585"] 

dfs = []
for model in models:
    jsons = sorted([str(p) for p in Path(model).rglob("*.json")])
    df = pd.concat([pd.read_json(j) for j in jsons], axis=0)
    df['model'] = model
    dfs.append(df)
    
df_meta = pd.concat(dfs, axis=0).reset_index(drop=True)
df = df_meta[(df_meta["type"]=="daily") & (df_meta['area'] > 250)].copy()
df['contours'] = df['projected'].apply(lambda x: np.array(x, dtype=np.int64).squeeze())
df = df[["name", "model", "contours"]]
df

,name,model,contours
2,a70243,GISS_E2_1_G_ssp585,"[[249, 28], [249, 29], [250, 30], [251, 30], [..."
6,131edb,GISS_E2_1_G_ssp585,"[[247, 28], [247, 29], [248, 29], [248, 29], [..."
7,54d00a,GISS_E2_1_G_ssp585,"[[249, 75], [249, 74], [247, 74], [247, 75], [..."
10,912e2a,GISS_E2_1_G_ssp585,"[[248, 25], [248, 26], [247, 26], [247, 28], [..."
12,09c1bc,GISS_E2_1_G_ssp585,"[[250, 78], [250, 78], [250, 78], [250, 78], [..."
...,...,...,...
44807,dd0618,GISS_E2_1_G_ssp585,"[[258, 39], [257, 39], [257, 38], [257, 38], [..."
44808,8b16d2,GISS_E2_1_G_ssp585,"[[260, 25], [260, 26], [259, 26], [259, 27], [..."
44820,43cc06,GISS_E2_1_G_ssp585,"[[295, 76], [296, 76], [296, 77], [296, 77], [..."
44821,22e483,GISS_E2_1_G_ssp585,"[[295, 65], [293, 65], [292, 65], [292, 65], [..."


In [4]:
####################################
# Create one template geo-image of Region-1
####################################

fp = f"Ext_max_t__Rgn_1__2020__Abv_Avg_5_K_for_3_days__CMIP6_GISS_E2_1_G_ssp585_Avg_yrs_1950_79.nc"

ds = xr.open_mfdataset(fp)

dr = pd.DatetimeIndex(ds['time'].dt.floor('D').values.astype('str'))
day = dr[0].strftime("%Y-%m-%d")

extreme = ds['extreme_yn'].sel(time=day)
x = extreme.squeeze()
zeros = xr.zeros_like(x)


In [5]:
####################################
# Generate barebone contour on geo-image of Region-1
####################################
%time

def create_fig(name, model, c):
    
    folder = f"figures/{model}"
    fp = f"{folder}/{name}.jpg"
    if not Path(fp).exists():
        
        fig, ax = plt.subplots(1,1, figsize=(10,10))

        zeros.copy().plot.imshow(ax=ax, add_colorbar=False)
        ax.axis('off')
        ax.set_title("");

        patch = [Polygon(c.squeeze(), True)]
        args = dict(edgecolors=("k",), linewidths=(1,), facecolor="none",)
        p = PatchCollection(patch, **args)
        ax.add_collection(p);

        if not os.path.exists(folder):
            os.mkdir(folder)
        fig.savefig(fp)
        fig.clear()
        plt.close(fig)

for i, (name, model, c) in df.iterrows():

    create_fig(name, model, c)
print('done!')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs


In [7]:
SAS_TOKEN = getpass.getpass() # of the whole "cmip6" folder in Azure.

 ·········································································································································


In [ ]:
####################################
# Upload images to Azure
####################################

URL_PREFIX = 'https://nasanex30analysis.blob.core.windows.net/cmip6'
models = ["GISS_E2_1_G_ssp585", "GISS_E2_1_G_ssp245", "GFDL_ESM4_ssp245", "GFDL_ESM4_ssp585"]

class AzureTarget():
    
    def __init__(self, filename):
        self.filename = filename
        
    def upload(self):
        
        sas_url = f"{URL_PREFIX}/{self.filename}?{SAS_TOKEN}"
        blob_client = BlobClient.from_blob_url(sas_url)
        
        with open(self.filename, "rb") as f:
            if blob_client.exists():
                warnings.warn(f"{self.filename} exists. Overwriting..")
            blob_client.upload_blob(f, overwrite=True)

for model in models:
    
    img_folder = f"shapes/{model}"

    figs = sorted([str(p) for p in Path(img_folder).rglob("*.jpg") if not "checkpoint" in str(p)])

    for fn in figs:
        AzureTarget(fn).upload()

    print('done!', model)

In [13]:
# model = "GISS_E2_1_G_ssp245"
# img_folder = f"figures/{model}"
# shutil.rmtree(img_folder)